# Set up the environment

In [1]:
!git clone https://github.com/WouterBant/diffusion-augmented-pixelsplat.git

Cloning into 'diffusion-augmented-pixelsplat'...
remote: Enumerating objects: 401, done.
remote: Counting objects: 100% (187/187), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 401 (delta 99), reused 125 (delta 78), pack-reused 214
Receiving objects: 100% (401/401), 2.98 MiB | 23.15 MiB/s, done.
Resolving deltas: 100% (136/136), done.


In [2]:
%cd diffusion-augmented-pixelsplat

/content/diffusion-augmented-pixelsplat


In [3]:
%pip install -r installation_jobs/requirements.txt > pip_install.log

  Running command git clone --filter=blob:none --quiet https://github.com/dcharatan/diff-gaussian-rasterization-modified /tmp/pip-req-build-4ku7dscu
  Running command git submodule update --init --recursive -q


Add a shortcut in your own Google Drive account to the Re10k sample data shared by the authors:
https://drive.google.com/drive/folders/1joiezNCyQK2BvWMnfwHJpm2V77c7iYGe

(This is done because it is easier to copy using the Google Drive - Colab integration than downloading from the public link, as Google Drive gives multiple HTML popups/warnings in that case that you'd need to process)

In [4]:
# Give permission when asked to access your drive from this notebook
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Download the empty folder that's currently checked into out repo, but which lacks the .torch files
!rm -rf "datasets/re10k"

# Copy the sample data from Google Drive to the local machine
!cp "/content/drive/MyDrive/dataset_subsets/re10k_subset.zip" "datasets/"

# Unzip
!unzip "datasets/re10k_subset.zip" -d "datasets"

# Rename to re10k
!mv "datasets/re10k_subset"  "datasets/re10k/"

Archive:  datasets/re10k_subset.zip
   creating: datasets/re10k_subset/
   creating: datasets/re10k_subset/train/
  inflating: datasets/re10k_subset/train/000002.torch  
  inflating: datasets/re10k_subset/train/000000.torch  
  inflating: datasets/re10k_subset/train/index.json  
  inflating: datasets/re10k_subset/train/000001.torch  
   creating: datasets/re10k_subset/test/
  inflating: datasets/re10k_subset/test/000001.torch  
  inflating: datasets/re10k_subset/test/index.json  
  inflating: datasets/re10k_subset/test/000000.torch  
  inflating: datasets/re10k_subset/test/000002.torch  


# Load images

In [50]:
from src.dataset.dataset_re10k import DatasetRE10k
from src.config import load_typed_root_config
from src.dataset.data_module import DataModule
from src.global_cfg import set_cfg
from hydra import compose, initialize_config_dir
from omegaconf import OmegaConf
import hydra
from torchvision.transforms import functional as F
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm.autonotebook import tqdm, trange

In [11]:
# All configs are stored in Hydra, a config management system, so we need to load these
# to instantiate classes like the data loader.
with initialize_config_dir(version_base=None, config_dir=hydra.utils.to_absolute_path("config")):
    cfg_dict = compose(
        config_name="main.yaml", return_hydra_config=True, overrides=[]
    )
    # print(OmegaConf.to_yaml(cfg))

cfg = load_typed_root_config(cfg_dict)
set_cfg(cfg)

In [12]:
# Instantiate the DataModule based on the configs and data
data_module = DataModule(
        cfg.dataset,
        cfg.data_loader,
)

In [62]:
!rm -rf {target_dir}

In [63]:
# Export all test images into a folder
target_dir = Path("datasets/re10k/re10k_test_context_images")
target_dir.mkdir(exist_ok=True)

for data in tqdm(data_module.test_dataloader()):
    # print(data.keys()) # ['context', 'target', 'scene']

    # context/image has a shape of [1, 2, 3, 180, 320], so if I understand correctly
    # context is a pair of images that is used for creating the reconstruction
    # print(data["context"].keys()) # ['extrinsics', 'intrinsics', 'image', 'near', 'far', 'index']

    # target/image has a shape of [1, 46, 3, 180, 320], so I assume target is a set of other images
    # from the same video that we try to reconstruct, based on the two images in context
    # print(data["target"].keys()) # ['extrinsics', 'intrinsics', 'image', 'near', 'far', 'index']

    # the scene id I presume
    # print(len(data["scene"])) # a single scene in a given batch, at least for the test set
    assert len(data["scene"]) == 1
    scene_id = data["scene"][0]

    image_group = "context" # context or target, save the context images for now
    images = data[image_group]["image"]
    # print(images.shape)
    for scene_num in range(images.shape[0]):
        for image_num in range(images[scene_num].shape[0]):
            pil_image = F.to_pil_image(images[scene_num, image_num])
            fn = f"{scene_id}_{image_group}_{image_num}.png"
            print(fn)
            pil_image.save(target_dir / fn, "PNG")
            # plt.imshow(pil_image)
            # plt.show()
            # break
        # break
    # break

  warnings.warn(_create_warning_msg(



  0%|          | 0/41 [00:00<?, ?it/s]

  self.pid = os.fork()



5aca87f95a9412c6_context_0.png
5aca87f95a9412c6_context_1.png
322261824c4a3003_context_0.png
322261824c4a3003_context_1.png
17d9303ee77c3a3d_context_0.png
17d9303ee77c3a3d_context_1.png
bc95e5c7e357f1b7_context_0.png
bc95e5c7e357f1b7_context_1.png
f7c0fa5b81552d35_context_0.png
f7c0fa5b81552d35_context_1.png
ffa95c3b40609c76_context_0.png
ffa95c3b40609c76_context_1.png
34b0658a5c200cdf_context_0.png
34b0658a5c200cdf_context_1.png
a56ba2efb5e3fdd9_context_0.png
a56ba2efb5e3fdd9_context_1.png
a47b88040452d7d9_context_0.png
a47b88040452d7d9_context_1.png
28e8300e004ab30b_context_0.png
28e8300e004ab30b_context_1.png
c48f19e2ffa52523_context_0.png
c48f19e2ffa52523_context_1.png
0d4de33c6888a754_context_0.png
0d4de33c6888a754_context_1.png
6558c5f10d45a929_context_0.png
6558c5f10d45a929_context_1.png
1214f2a11a9fc1ed_context_0.png
1214f2a11a9fc1ed_context_1.png
41bcd011f99bfb66_context_0.png
41bcd011f99bfb66_context_1.png
89ea49cd9865aeff_context_0.png
89ea49cd9865aeff_context_1.png
a9b3ff60

In [64]:
!cp -r {target_dir} "/content/drive/MyDrive/Shared/"

In [65]:
!git fetch
!git status

On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	datasets/re10k/re10k_test_context_images/
	datasets/re10k_subset.zip

nothing added to commit but untracked files present (use "git add" to track)
